In [39]:
import numpy as np
import pandas as pd
from template import *
import itertools
from datetime import datetime

# Exercise 1

# Exercise 2

## Exercise 2.1

In [42]:
def eclat(db, minsup):
    start_time=datetime.now()
    time_format="%H:%M:%S:%f"
    temporal_items_list = []
    for t in db:
        temporal_items_list.extend(t)
    temporal_items_list = list(sorted(set(temporal_items_list)))
    item_list = temporal_items_list.copy()
    item_list_transactions = {item: [] for item in temporal_items_list}
    for transaction_index, t in enumerate(db):
        for transaction_item in t:
            item_list_transactions[(transaction_item)].append(transaction_index + 1)
    itemset_supports = [([item], item_list_transactions[item]) for item in item_list]
    for itemset in itemset_supports:
        if len(itemset[1]) < minsup:
            item_list.remove(itemset[0][0])
            itemset_supports.remove(itemset)
    final_supports_list=supportsNextLevel(itemset_supports,itemset_supports, item_list, minsup)
    final_supports=[(item,len(transactions)) for item,transactions in final_supports_list]
    final_time=datetime.now()
    execution_time=(final_time-start_time)
    print(f'Runtime: {execution_time}\n	Start Time: {start_time.strftime(time_format)}\n	Finish Time: {final_time.strftime(time_format)}')
    return(final_supports)
    
def supportsNextLevel(total_supports,supports, items, minsup, first_time=True):
    for index, itemset in enumerate(items[:-1]):
        level_supports = []
        itemset_derivate_list = []
        items_to_combine = items[index+1:]
        for other_itemset in items_to_combine:
            new_itemset_list = sorted(list(set(itemset + other_itemset)))
            if first_time:
                new_itemset_support = list(set(findSupport(supports, [itemset])).intersection(findSupport(supports, [other_itemset])))
            else:
                new_itemset_support = list(set(findSupport(supports, itemset)).intersection(findSupport(supports, other_itemset)))
            if len(new_itemset_support) >= minsup:
                level_supports.append((new_itemset_list, new_itemset_support))
                itemset_derivate_list.append(new_itemset_list)
        total_supports.extend(level_supports)
        if len(level_supports) > 1 and level_supports:
            supportsNextLevel(total_supports,level_supports, itemset_derivate_list, minsup, False)
    if first_time:
        return(total_supports)

def findSupport(supports,itemset):
    for support in supports:
        if itemset==support[0]:
            return support[1]

In [43]:
#Execution With Shop Dataset and 500 Minsups
dbShop= read_database("shop.dat")
itemset_supports = eclat(dbShop, 500)
column_names=["Itemset", "Support"]

Runtime: 0:00:36.422068
	Start Time: 07:44:00:398017
	Finish Time: 07:44:36:820085


In [44]:
df_itemset_supports = pd.DataFrame(itemset_supports,columns=column_names)
display(df_itemset_supports)

,Itemset,Support
0,[0],594
1,[1],1535
2,[10],1351
3,[100],1749
4,[102],109
...,...,...
1151,"[3, 5, 8, 9]",535
1152,"[6, 8, 9]",810
1153,"[1, 4, 8, 9]",546
1154,"[2, 3, 4, 7, 9]",859


### test

In [45]:
## Unit test
import pandas as pd
dbExample = read_database("example.dat")

def testSyntax(F):
    found_problem = False
    for t in F:
        if type(t) != tuple:
            print("Entry " + str(t) + " in frequent itemset collection F is not a tuple.")
            found_problem = True
        elif len(t) != 2:
            print("Entry " + str(t) + " in frequent itemset collection F is a tuple but of length " + str(len(t)) + " instead of 2.")
            found_problem = True
        elif type(t[0]) != list:
            print("Itemset type in entry " + str(t) + " is " + str(type(t[0])) + " but should be list!")
            found_problem = True
        elif type(t[1]) != int:
            print("Support in entry " + str(t) + " is of type " + str(type(t[1])) + " but should be int!")
            found_problem = True
    print("Syntax check on frequent itemset collection: " + ("FAILED" if found_problem else "OK"))

def testFrequentItemsetCollection(F_exp, F_act):
    testSyntax(F_act)
    length_exp = len(F_exp)
    print(length_exp)
    length_act = len(F_act)
    print("Length of collection: " + str("OK" if length_exp == length_act else "FAILED, expected length " + str(length_exp) + " but saw " + str(length_act)))
    obsolete = [f for f in F_act if not f in F_exp]
    missing = [f for f in F_exp if not f in F_act]
    if obsolete:
        print("Found unexpected entries in F: " + str(obsolete))
    if missing:
        print("Missing entries in F: " + str(missing))
    if not missing and not obsolete:
        print("F seems to be correct.")

F_exp = [(['A'], 4), (['A', 'B'], 4), (['A', 'B', 'D'], 3), (['A', 'B', 'D', 'E'], 3), (['A', 'B', 'E'], 4), (['A', 'D'], 3), (['A', 'D', 'E'], 3), (['A', 'E'], 4), (['B'], 6), (['B', 'C'], 4), (['B', 'C', 'E'], 3), (['B', 'D'], 4), (['B', 'D', 'E'], 3), (['B', 'E'], 5), (['C'], 4), (['C', 'E'], 3), (['D'], 4), (['D', 'E'], 3), (['E'], 5)]

# Test ECLAT
F_act = eclat(dbExample, 3)

testFrequentItemsetCollection(F_exp, F_act)

Runtime: 0:00:00
	Start Time: 07:45:28:418321
	Finish Time: 07:45:28:418321
Syntax check on frequent itemset collection: OK
19
Length of collection: OK
F seems to be correct.


In [ ]:
def testRuleGeneration(R_exp, R_act):

    # test syntax
    found_problem = False
    for r in R_act:
        if type(r) != tuple:
            print("Rule " + str(r) + " is not represented as a tuple!")
            found_problem = True
        if len(r) != 4:
            print("Rule " + str(r) + " does not consist of 4 entries. Should be (premise, conclusion, (absolute) support, confidence)")
            found_problem = True
        if type(r[0]) != list:
            print("Premise of rule " + str(r) + " is of type " + str(type(r[0])) + " but should be list!")
            found_problem = True
        if type(r[1]) != list:
            print("Conclusion of rule " + str(r) + " is of type " + str(type(r[0])) + " but should be list!")
            found_problem = True
    print("Syntax check " + ("FAILED. Skipping rest of the test." if found_problem else "OK"))
    if found_problem:
        return
    
    # semantics test
    length_exp = len(R_exp)
    length_act = len(R_act)
    print("Length of rule set: " + str("OK" if length_exp == length_act else "FAILED, expected length " + str(length_exp) + " but saw " + str(length_act)))
    obsolete = [f for f in R_act if not f in R_exp]
    missing = [f for f in R_exp if not f in R_act]
    if obsolete:
        print("Found unexpected entries in R: " + str(obsolete))
        found_problem = True
    if missing:
        print("Missing entries in R: " + str(missing))
        found_problem = True
    print("Rule set test: " + ("FAILED" if found_problem else "OK"))

R_exp = [(['A'], ['B'], 4, 1.0), (['A', 'D'], ['B'], 3, 1.0), (['B', 'D', 'E'], ['A'], 3, 1.0), (['A', 'D', 'E'], ['B'], 3, 1.0), (['A', 'B', 'D'], ['E'], 3, 1.0), (['D', 'E'], ['A', 'B'], 3, 1.0), (['A', 'D'], ['B', 'E'], 3, 1.0), (['A', 'E'], ['B'], 4, 1.0), (['A', 'B'], ['E'], 4, 1.0), (['A'], ['B', 'E'], 4, 1.0), (['D', 'E'], ['A'], 3, 1.0), (['A', 'D'], ['E'], 3, 1.0), (['A'], ['E'], 4, 1.0), (['C'], ['B'], 4, 1.0), (['C', 'E'], ['B'], 3, 1.0), (['D'], ['B'], 4, 1.0), (['D', 'E'], ['B'], 3, 1.0), (['E'], ['B'], 5, 1.0)]
R_act = getStrongRulesForDatabase(dbExample, 3, 1.0)
testRuleGeneration(R_exp, R_act)

NameError: name 'getStrongRulesForDatabase' is not defined